In [40]:
# general lib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# fuzzy
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# time ani
import time

In [41]:
# data and smol klining
df = pd.read_csv("BBCA.csv")

df1 = df.iloc[4882:5145]
print(df1.head())
print(df1.tail())

df_pakai = df1['close']
print(df_pakai)
df_pakai = df_pakai.values.tolist()
print(df_pakai)

       timestamp  open   low  high  close    volume
4882  2020-01-01  6680  6670  6720   6685         0
4883  2020-01-02  6695  6680  6780   6690  49445000
4884  2020-01-03  6750  6725  6800   6800  47755500
4885  2020-01-06  6720  6690  6750   6735  27300000
4886  2020-01-07  6735  6730  6770   6740  45022500
       timestamp  open   low  high  close    volume
5140  2020-12-28  6790  6660  6820   6780  60234000
5141  2020-12-29  6775  6740  6810   6765  34212500
5142  2020-12-30  6800  6700  6800   6770  52675500
5143  2020-12-31  6800  6700  6800   6770         0
5144  2021-01-01  6800  6700  6800   6770         0
4882    6685
4883    6690
4884    6800
4885    6735
4886    6740
        ... 
5140    6780
5141    6765
5142    6770
5143    6770
5144    6770
Name: close, Length: 263, dtype: int64
[6685, 6690, 6800, 6735, 6740, 6680, 6740, 6725, 6745, 6870, 6835, 6850, 6875, 6835, 6830, 6840, 6840, 6810, 6840, 6790, 6785, 6740, 6480, 6440, 6600, 6730, 6740, 6760, 6785, 6780, 6800, 6790, 6

In [42]:
# inisiasi global dan lokal

df_global = []
i = 0
while i < 100:
    df_global.append(df_pakai[i])
    i += 1
print(df_global)

df_lokal = []
i = 30
while i > 0:
    df_lokal.append(df_global[len(df_global)-i])
    i -= 1
    
print(df_lokal)

[6685, 6690, 6800, 6735, 6740, 6680, 6740, 6725, 6745, 6870, 6835, 6850, 6875, 6835, 6830, 6840, 6840, 6810, 6840, 6790, 6785, 6740, 6480, 6440, 6600, 6730, 6740, 6760, 6785, 6780, 6800, 6790, 6680, 6720, 6695, 6695, 6595, 6615, 6525, 6530, 6420, 6290, 6290, 6080, 6320, 6440, 6435, 6200, 5785, 5925, 5850, 5560, 5560, 5560, 5120, 5000, 4650, 4735, 4430, 4500, 4500, 5280, 5510, 5495, 5525, 5480, 5410, 5495, 5735, 5655, 5635, 5595, 5595, 5500, 5515, 5485, 5240, 5425, 5355, 5160, 5145, 5120, 4920, 4960, 4830, 4850, 5170, 5170, 5220, 5285, 5355, 5355, 5245, 5295, 5220, 5110, 4920, 4785, 4765, 4680]
[5635, 5595, 5595, 5500, 5515, 5485, 5240, 5425, 5355, 5160, 5145, 5120, 4920, 4960, 4830, 4850, 5170, 5170, 5220, 5285, 5355, 5355, 5245, 5295, 5220, 5110, 4920, 4785, 4765, 4680]


In [43]:
# hasil fuzzy

fuzzy = []

# loop

i = len(df_global)
while i < len(df_pakai):
    
    # update dulu
    df_global.append(df_pakai[i])
    del df_lokal[0]
    df_lokal.append(df_global[-1])
    
    # fibonacci
    max_global = max(df_global)
    min_global = min(df_global)
    fib_global = abs(df_global[-1]-0.618*(max_global-min_global))/(max_global-0.618*(max_global-min_global))*100
    
    max_lokal = max(df_lokal)
    min_lokal = min(df_lokal)
    fib_lokal = abs(df_lokal[-1]-0.618*(max_lokal-min_lokal))/(max_lokal-0.618*(max_lokal-min_lokal))*100
    
    # fuzzy logic
    # Create the fuzzy variables and membership functions
    fibonacci_retracement_global = ctrl.Antecedent(np.arange(0, 101, 1), 'fibonacci_retracement_global')
    fibonacci_retracement_lokal = ctrl.Antecedent(np.arange(0, 101, 1), 'fibonacci_retracement_lokal')
    decision = ctrl.Consequent(np.arange(0, 101, 1), 'decision')
    
    
    # Define the membership functions for the variables
    fibonacci_retracement_global['close'] = fuzz.trimf(fibonacci_retracement_global.universe, [0, 0, 50])
    fibonacci_retracement_global['medium'] = fuzz.trimf(fibonacci_retracement_global.universe, [30, 50, 70])
    fibonacci_retracement_global['far'] = fuzz.trimf(fibonacci_retracement_global.universe, [50, 100, 100])
    
    fibonacci_retracement_lokal['close'] = fuzz.trimf(fibonacci_retracement_lokal.universe, [0, 0, 50])
    fibonacci_retracement_lokal['medium'] = fuzz.trimf(fibonacci_retracement_lokal.universe, [30, 50, 70])
    fibonacci_retracement_lokal['far'] = fuzz.trimf(fibonacci_retracement_lokal.universe, [50, 100, 100])
    
    decision['sell'] = fuzz.trimf(decision.universe, [0, 0, 50])
    decision['neutral'] = fuzz.trimf(decision.universe, [0, 50, 100])
    decision['buy'] = fuzz.trimf(decision.universe, [50, 100, 100])
    
    # Define the fuzzy rules to be updated as times goes on
    rules = [
        ctrl.Rule(fibonacci_retracement_global['close'] & fibonacci_retracement_lokal['close'], decision['buy']),
        ctrl.Rule(fibonacci_retracement_global['close'] & fibonacci_retracement_lokal['medium'], decision['buy']),
        ctrl.Rule(fibonacci_retracement_global['close'] & fibonacci_retracement_lokal['far'], decision['neutral']),
        
        ctrl.Rule(fibonacci_retracement_global['medium'] & fibonacci_retracement_lokal['close'], decision['buy']),
        ctrl.Rule(fibonacci_retracement_global['medium'] & fibonacci_retracement_lokal['medium'], decision['neutral']),
        ctrl.Rule(fibonacci_retracement_global['medium'] & fibonacci_retracement_lokal['far'], decision['sell']),
        
        ctrl.Rule(fibonacci_retracement_global['far'] & fibonacci_retracement_lokal['close'], decision['neutral']),
        ctrl.Rule(fibonacci_retracement_global['far'] & fibonacci_retracement_lokal['medium'], decision['sell']),
        ctrl.Rule(fibonacci_retracement_global['far'] & fibonacci_retracement_lokal['far'], decision['sell'])
    ]
    
    # Create the fuzzy control system
    decision_ctrl = ctrl.ControlSystem(rules)
    decision_calc = ctrl.ControlSystemSimulation(decision_ctrl)
    
    # Pass inputs and compute the decision
    decision_calc.input['fibonacci_retracement_global'] = fib_global
    decision_calc.input['fibonacci_retracement_lokal'] = fib_lokal
    
    #cek kadal
    print(fib_global, ":::", fib_lokal, "\n")
    
    # Compute the decision and place it in list
    decision_calc.compute()
    fuzzy.append(decision_calc.output['decision'])
    
    # loop control
    i += 1

60.66361048398673 ::: 83.49746397774742 

60.66361048398673 ::: 83.49746397774742 

60.66361048398673 ::: 84.99690936332884 

60.66361048398673 ::: 84.99690936332884 

64.48539240378896 ::: 89.67420616700518 

64.39217821062306 ::: 90.73438088543062 

70.544314959573 ::: 97.38145546762169 

68.58681690308893 ::: 96.65846471642516 

68.58681690308893 ::: 96.65846471642516 

71.75609947072981 ::: 100.0 

79.5860916966661 ::: 100.0 

79.77252008299793 ::: 100.0 

78.56073557184111 ::: 98.72649401844036 

81.8232323326479 ::: 100.0 

80.05216266249565 ::: 98.1539202959946 

79.95894846932974 ::: 98.05675820631009 

77.72180783334794 ::: 95.7248680538822 

77.53537944701611 ::: 95.53054387451321 

74.36609687937525 ::: 92.22703282524037 

79.21323492400246 ::: 97.27946148883413 

78.4675213786752 ::: 96.50216477135817 

75.95073816319568 ::: 93.8787883498768 

75.76430977686385 ::: 93.68446417050781 

75.11181042470251 ::: 93.00432954271635 

76.60323751535704 ::: 94.55892297766827 

78.187

In [47]:
if(df_global == df_pakai):
    print("ara ara")

ara ara


In [48]:
print(fuzzy)

[19.75699063589763, 19.75699063589763, 19.75699063589763, 19.75699063589763, 21.583126314779218, 21.603586938989114, 20.306588218574838, 20.706913527364176, 20.706913527364176, 20.065090960067394, 18.63941922014287, 18.60869346432609, 18.81134188087256, 18.282050196556483, 18.562918833947418, 18.57813495408525, 18.955590574585834, 18.988071570827476, 19.56262100321864, 18.701367182885164, 18.827212684894487, 19.2701902334954, 19.304071392193865, 19.423763541571258, 19.152736788525836, 18.875061866639868, 18.81134188087256, 19.06993750998664, 18.971811970544483, 18.907157096512904, 18.57813495408525, 18.36902256810576, 18.36902256810576, 18.182664730006465, 18.031170708908917, 17.510093933437826, 17.746347618288397, 17.510093933437826, 17.56702192347603, 17.510093933437826, 17.625394426913463, 17.55551973779376, 17.69730380675361, 17.64914078994463, 17.510093933437826, 17.55551973779376, 17.510093933437826, 17.746347618288397, 17.746347618288397, 17.544075586634197, 17.66109812507326, 1

In [46]:
print(df_global)
print(df_pakai)
print(df_lokal)

[6685, 6690, 6800, 6735, 6740, 6680, 6740, 6725, 6745, 6870, 6835, 6850, 6875, 6835, 6830, 6840, 6840, 6810, 6840, 6790, 6785, 6740, 6480, 6440, 6600, 6730, 6740, 6760, 6785, 6780, 6800, 6790, 6680, 6720, 6695, 6695, 6595, 6615, 6525, 6530, 6420, 6290, 6290, 6080, 6320, 6440, 6435, 6200, 5785, 5925, 5850, 5560, 5560, 5560, 5120, 5000, 4650, 4735, 4430, 4500, 4500, 5280, 5510, 5495, 5525, 5480, 5410, 5495, 5735, 5655, 5635, 5595, 5595, 5500, 5515, 5485, 5240, 5425, 5355, 5160, 5145, 5120, 4920, 4960, 4830, 4850, 5170, 5170, 5220, 5285, 5355, 5355, 5245, 5295, 5220, 5110, 4920, 4785, 4765, 4680, 4765, 4765, 4765, 4765, 4970, 4965, 5295, 5190, 5190, 5360, 5780, 5790, 5725, 5900, 5805, 5800, 5680, 5670, 5500, 5760, 5720, 5585, 5575, 5540, 5620, 5705, 5725, 5645, 5675, 5695, 5800, 5870, 5870, 5935, 5990, 6200, 6100, 6200, 6175, 6200, 6150, 6180, 6120, 6140, 6200, 6180, 6200, 6100, 6100, 6185, 6135, 6240, 6240, 6130, 6210, 6205, 6260, 6180, 6120, 6175, 6260, 6300, 6405, 6405, 6360, 6330, 633